Set Cover problem

In [1]:
from random import random, seed
from itertools import product
import numpy as np
from matplotlib import pyplot as plt
from icecream import ic
from tqdm import tqdm

In [2]:
#Helper Functions
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    
    return np.all(np.logical_or.reduce(SETS[solution]))
def fitness(solution: np.array) -> tuple:
    """Returns a tuple (validity, negative cost) to maximize validity and minimize cost."""
    return (valid(solution), -cost(solution))

def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()


In [3]:
universes = [100, 1000, 10_000, 100_000, 100_000, 100_000]
num_sets =[10, 100, 1000, 10_000, 10_000, 10_000]
densities= [0.2, 0.2, 0.2, 0.1, 0.2, 0.3]

In [7]:
def single_tweak(solution, num_elements):
    new_solution = solution.copy()
    flip_index = np.random.randint(num_elements)

    new_solution[flip_index] = not new_solution[flip_index] 
    pass
def multiple_tweak(solution, num_sets):
    new_solution = solution.copy()
    index = None
    while index is None or np.random.random() < 0.4:
        index = np.random.randint(0, num_sets)
        new_solution[index] = not new_solution[index]
    return new_solution
def simulated_annealing(set_data, cost_values, num_elements, density, rng, start_temp=900.0, decay_rate=0.995, end_temp=1e-6, max_steps=1000,mode="single"):
    if num_elements < 1000:
        current_state = rng.random(num_elements) < 1
    else:
        current_state = rng.random(num_elements) < 0.5
    # Evaluate the initial solution
    current_score = fitness(current_state)
    optimal_state = current_state
    optimal_score = current_score
    optimal_step = 0
    # Set initial temperature and track history
    temp = start_temp
    fitness_history = [current_score[1]]
    validity_history = [current_score[0]]
    # Begin simulated annealing loop
    for step in tqdm(range(max_steps), desc="Simulated Annealing"):
        # Generate a candidate solution by tweaking the current one
        if mode == "single":
            candidate_state = single_tweak(current_state,num_elements)

        elif mode == "multiple":
            candidate_state = multiple_tweak(current_state, num_elements)


        candidate_score = fitness(candidate_state)

        # Check if the new solution is valid before proceeding
        if not valid(candidate_state):
            validity_history.append(False)
        else:
            validity_history.append(True)
        
        # Calculate the acceptance probability according to the Metropolis criterion
        score_diff = current_score[1] - candidate_score[1]
        acceptance_prob = np.exp(-score_diff / temp)

        # Move to the candidate solution if it is better or with a given probability
        if (candidate_score > current_score or rng.random() < acceptance_prob):# and not (valid(current_state)==False and valid(candidate_state)==False):
        
            current_state = candidate_state
            current_score = candidate_score

            # Update the best solution if this one is better
            if current_score > optimal_score:
                optimal_state = current_state
                optimal_score = current_score
                optimal_step = step
        # Gradually decrease the temperature
        temp = max(temp * decay_rate, end_temp)

        # Track the progress of the optimization
        fitness_history.append(current_score[1])
        # Stop if the temperature has cooled down to the minimum
        if temp <= end_temp:
            break

        ic(step, optimal_score, temp)
    print(fitness_history)
    return optimal_step,optimal_state, optimal_score, fitness_history,validity_history

In [8]:
#given list of solutions and list of validities, get list of invalid solution and list of indexes
def get_invalid_solutions(solutions, validities):
    invalid_solutions = []
    invalid_indexes = []
    for i in range(len(solutions)):
        if not validities[i]:
            invalid_solutions.append(solutions[i])
            invalid_indexes.append(i)
    return invalid_solutions, invalid_indexes


In [ ]:
for universe, num_set, density in zip(universes, num_sets, densities):
    print(f"Universe Size: {universe}, Num Sets: {num_set}, Density: {density}")
    
    rng = np.random.Generator(np.random.PCG64([universe, num_set, int(10_000 * density)]))
    
    SETS = rng.random((num_set, universe)) < density

    
    for s in range(universe):
        if not np.any(SETS[:, s]):
            SETS[np.random.randint(num_set), s] = True

    #for each set a cost proportional to the number of element covered is associated to it
    COSTS = np.power(SETS.sum(axis=1), 1.1)
    
    best_step,sol_val, sol_cost, hist,val_hist = simulated_annealing(SETS, COSTS, num_set, density, rng, mode="multiple")
    
    print(f"Best cost for: {universe}: {sol_cost}")
    #print(f"Best validity solution: {sol_val}")

    plt.figure(figsize=(14, 8))
    #plot the fitness history
    plt.plot(range(len(hist)), hist, color="blue", label="Fitness")
    #plot invalid solutions
    invalid_solutions, invalid_indexes = get_invalid_solutions(hist, val_hist)
    plt.scatter(invalid_indexes, invalid_solutions, color="red", marker="x", label="invalid solutions")
    #plot best solution
    print(f"Best step: {best_step}")
    print(f"Best solution: {sol_cost}")
    plt.scatter(best_step, sol_cost[1], color="green", marker="o", label="Best solution (valid)")
    plt.title(f" Universe size: {universe}")
    plt.xlabel("Steps")
    plt.ylabel("Negative cost (fitness)")
    plt.legend()
    plt.grid(True)
    plt.show()
    

In [ ]:
random()